## Stochastic Thinking and Random Walks
随机思维

The world may or may not be inherently unpredicatable  
But our lack of knowledge does not allow us to make accurate predictions  
Therefore we might as well threat the world as inherently unpredictable  
**Predictive nondeterminism**

### Implementing a Randomo Process

In [1]:
import random 

def rollDie():
    '''returns a random int between 1 and 6'''
    return random.choice([1,2,3,4,5,6])

def testRoll(n=10):
    result = ''
    for i in range(n):
        result = result + str(rollDie())
    print(result)

testRoll()

4551526651


### A Simulation of Die Rolling

In [5]:
def runSim(goal, numTrials, txt):
    total = 0
    for i in range(numTrials):
        result = ''
        for j in range(len(goal)):
            result += str(rollDie())
        if result == goal:
            total += 1
    print('Actual probability of',txt,'=',round(1/(6**len(goal)),8))
    estProbability = round(total/numTrials,8)
    print('Estimated Probability of',txt,'=',round(estProbability,8))
    
runSim('11111',1000000,'11111')

Actual probability of 11111 = 0.0001286
Estimated Probability of 11111 = 0.000112


### Approximating Using a Simulation

In [9]:
def sameDate(numPeople, numSame):
    possibleDates = range(366)
    birthdays = [0]*366
    for p in range(numPeople):
        birthDate = random.choice(possibleDates)
        birthdays[birthDate] += 1
    return max(birthdays) >= numSame

In [10]:
def birthdayProb(numPeople, numSame, numTrials):
    numHits = 0
    for t in range(numTrials):
        if sameDate(numPeople, numSame):
            numHits += 1
    return numHits/numTrials

import math

for numPeople in [10, 20, 40, 100]:
    print('For', numPeople,
          'est. prob. of a shared birthday is',
          birthdayProb(numPeople, 2, 10000))
    numerator = math.factorial(366)
    denom = (366**numPeople)*math.factorial(366-numPeople)
    print('Actual prob. for N = 100 =',
          1 - numerator/denom)

For 10 est. prob. of a shared birthday is 0.116
Actual prob. for N = 100 = 0.1166454118039999
For 20 est. prob. of a shared birthday is 0.41
Actual prob. for N = 100 = 0.4105696370550831
For 40 est. prob. of a shared birthday is 0.895
Actual prob. for N = 100 = 0.89054476188945
For 100 est. prob. of a shared birthday is 1.0
Actual prob. for N = 100 = 0.9999996784357714


### But All Dates Are Not Equally Likely

![birthdaydate](./img/BirthdayDate.png)

## Random Walks

#### Random walks are important in many domains
Understanding the stock market(maybe)  
Modeling diffusion processes  
Etc.
#### Good illustration of how to use simulation to understand things

## Structure of Simulation
### 1.Simulate one walks of k steps
### 2.Simulate n such walks
### 3.Report average distance from origin

Location: a place  
Field: a collection of places and drunks  
Drunk: somebody who wanders from place to place in a field

In [11]:
class Location(object):
    def __init__(self, x, y):
        '''x and y are floats'''
        self.x = x
        self.y = y
    
    def move(self, deltax, deltay):
        '''deltax and dletay are floats'''
        return Location(self.x+deltax, self.y+deltay)
    
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def distFrom(self, other):
        xDist = self.x - other.getX()
        yDist = self.y - other.getY()
        return (xDist**2 + yDist**2)**0.5
    
    def __str__(self):
        return '<' + str(self.x) + ',' + str(self.y) + '>'

In [13]:
class Drunk(object):
    def __init__(self, name=None):
        '''Assume name is str'''
        self.name = name
    
    def __str__(self):
        if self != None:
            return self.name
        return 'Anonymous'

In [14]:
import random

class UsualDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0,1),(0,-1),(1,0),(-1,0)]
        return random.choice(stepChoices)
    
class MasochistDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0.0,1.1),(0.0,-0.9),(1.0,0.0),(-1.0,0.0)]
        return random.choice(stepChoices)

In [15]:
class Field(object):
    def __init__(self):
        self.drunks = {}
    
    def addDrunks(self, drunk, loc):
        if drunk in self.drunks:
            raise ValueError('Duplicate drunk')
        else:
            self.drunks[drunk] = loc
    
    def getLoc(self,drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        return self.drunks[drunk]
    
    def moveDrunk(self, drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        xDist, yDist = drunk.takeStep()
        # use move method of location to get new location
        self.drunks[drunk] = self.drunks[drunk].move(xDist,yDist)

## Simulating a Single Walk

In [16]:
def walk(f,d,numSteps):
    '''Assume: f a Field, d a Drunk in f, and numSteps an int >= 0.
       Moves d numSteps times; returns the distance between the final
       location and the location at the start of the walk.'''
    start = f.getLoc(d)
    for s in range(numSteps):
        f.moveDrunk(d)
    return start.distFrom(f.getLoc(d))

## Simulating Multiple Walks

In [31]:
def simWalks(numSteps, numTrials, dClass):
    '''Assumes numSteps an int >= 0, numTrials an int > 0, dClass a 
       subclass of Drunk Simulates numTrials walks fo numSteps steps
       each. Returns a list of the final distances for each trial.'''
    Homer = dClass('Homer')
    origin = Location(0,0)
    distances = []
    for t in range(numTrials):
        f = Field()
        f.addDrunks(Homer,origin)
        distances.append(round(walk(f,Homer,numSteps),1))
        
    return distances

In [32]:
def drunkTest(walkLengths, numTrials, dClass):
    '''Assumes walkLengths a sequence of ints >= 0
       numTrials an int > 0, dClass a subclass of Drunk.
       For each number of steps in walkLenghts, runs simWalks with
       numTriasls walk and prints results'''
    for numSteps in walkLengths:
        distances = simWalks(numSteps, numTrials, dClass)
        print(dClass.__name__,'random walk of',numSteps,'steps')
        print(' Mean =', round(sum(distances)/len(distances),4))
        print(' Max =', max(distances),'Min =',min(distances))

### Sanity Check

In [34]:
drunkTest((0,1,2), 100, UsualDrunk)

UsualDrunk random walk of 0 steps
 Mean = 0.0
 Max = 0.0 Min = 0.0
UsualDrunk random walk of 1 steps
 Mean = 1.0
 Max = 1.0 Min = 1.0
UsualDrunk random walk of 2 steps
 Mean = 1.232
 Max = 2.0 Min = 0.0


In [33]:
drunkTest((10,100,1000,10000),100,UsualDrunk)

UsualDrunk random walk of 10 steps
 Mean = 2.751
 Max = 6.3 Min = 0.0
UsualDrunk random walk of 100 steps
 Mean = 8.342
 Max = 22.1 Min = 1.4
UsualDrunk random walk of 1000 steps
 Mean = 27.576
 Max = 77.9 Min = 4.0
UsualDrunk random walk of 10000 steps
 Mean = 88.393
 Max = 208.2 Min = 6.0


In [36]:
def simAll(drunkKinds, walkLengths, numTrials):
    for dClass in drunkKinds:
        drunkTest(walkLengths, numTrials, dClass)

simAll((UsualDrunk, MasochistDrunk), (1000, 10000), 100)

UsualDrunk random walk of 1000 steps
 Mean = 27.242
 Max = 74.1 Min = 4.2
UsualDrunk random walk of 10000 steps
 Mean = 87.762
 Max = 232.2 Min = 3.2
MasochistDrunk random walk of 1000 steps
 Mean = 58.01
 Max = 117.7 Min = 14.2
MasochistDrunk random walk of 10000 steps
 Mean = 501.878
 Max = 693.6 Min = 343.9
